# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# import packages and modifying output settings
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Remember to enter in the cell twice for this to work

# Loading files
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Creating Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Merging datasets into one DataFrame
school_data_complete = pd.merge(student_data, school_data, how="left", 
                                on=["school_name", "school_name"])

# Removing unnecessary columns
school_data_analysis = school_data_complete[['school_name','Student ID','grade',
                                             'reading_score','math_score',
                                             'School ID', 'type','size','budget']]
# Initial overview of DataFrame
school_data_analysis.head(3)

# Printing a list of column names for reference
list(school_data_analysis)

# stylistic note: _calc and _pre are my interim step variables 
# to make the variables I need 

['school_name',
 'Student ID',
 'grade',
 'reading_score',
 'math_score',
 'School ID',
 'type',
 'size',
 'budget']

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# Total number of schools
school_total = school_data_analysis['school_name'].nunique()
school_total

15

In [3]:
# Total number of students
student_total = school_data_analysis['Student ID'].count()
student_total

39170

In [4]:
# Total budget
district_total_budget = sum(school_data_analysis['budget'].unique())
district_total_budget

24649428

In [5]:
# Average math score 
avg_math_score = round(school_data_analysis['math_score'].mean(),2)
avg_math_score

78.99

In [6]:
# Average reading score
avg_read_score = round(school_data_analysis['reading_score'].mean(),2)
avg_read_score

81.88

In [7]:
# Overall passing rate 
overall_pass_rate = round(((school_data_analysis['math_score'].mean()) + 
                           (school_data_analysis['reading_score'].mean())) / 2, 2)
overall_pass_rate

80.43

In [8]:
# Percentage of students with a passing math score (70 or greater)
# .tolist() was useful here
math_pass_calc = round(((school_data_analysis['math_score'] >= 70)
                        .value_counts("True")) * 100, 2).tolist()
math_pass = math_pass_calc[0]
math_pass

74.98

In [9]:
# Percentage of students with a passing reading score (70 or greater)
reading_pass_calc = round(((school_data_analysis['reading_score'] >= 70)
                           .value_counts("True")) * 100, 2).tolist()
reading_pass = reading_pass_calc[0]
reading_pass

85.81

In [42]:
# Creating dataframe with the above datapoints

# school_data_analysis["district_total_budget"] = school_data_analysis["district_total_budget"].map("${:.2f}".format)

dataSummary1 = pd.DataFrame({'School Total': school_total, 'Student Total': student_total,
                            'District Total Budget': district_total_budget,'Average Math Score':
                            avg_math_score,'Average Reading Score': avg_read_score,
                            'Overall Passing Rate': overall_pass_rate, 'Math Passing Rate (score >= 70)':
                            math_pass, 'Reading Passing Rate (score >= 70)': reading_pass},index=[0])

dataSummary1

KeyError: 'district_total_budget'

## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [27]:
school_data_analysis.head(2)

,school_name,Student ID,grade,reading_score,math_score,School ID,type,size,budget
0,Huang High School,0,9th,66,79,0,District,2917,1910635
1,Huang High School,1,12th,94,61,0,District,2917,1910635


In [35]:
total_students = school_data_analysis['school_name'].value_counts()
total_students

type(byschool_summary)

# byschool_summary_df = byschool_summary.to_dict()
# byschool_summary_df

# type(byschool_summary_df)

Bailey High School       4976
Johnson High School      4761
Hernandez High School    4635
Rodriguez High School    3999
Figueroa High School     2949
Huang High School        2917
Ford High School         2739
Wilson High School       2283
Cabrera High School      1858
Wright High School       1800
Shelton High School      1761
Thomas High School       1635
Griffin High School      1468
Pena High School          962
Holden High School        427
Name: school_name, dtype: int64

pandas.core.series.Series

In [40]:
# Making a list of budgets

byschool_summary = school_data_analysis.groupby(['school_name']).mean()['budget']
byschool_summary

school_name
Bailey High School       3124928.0
Cabrera High School      1081356.0
Figueroa High School     1884411.0
Ford High School         1763916.0
Griffin High School       917500.0
Hernandez High School    3022020.0
Holden High School        248087.0
Huang High School        1910635.0
Johnson High School      3094650.0
Pena High School          585858.0
Rodriguez High School    2547363.0
Shelton High School      1056600.0
Thomas High School       1043130.0
Wilson High School       1319574.0
Wright High School       1049400.0
Name: budget, dtype: float64

In [13]:
# # Getting the school name, school type, and total 
# # of students as a series and converting to a DataFrame

# total_students_pre = byschool_summary.size()
# total_students_pre

# total_students = pd.DataFrame(total_students_pre)
# total_students

In [ ]:
# Getting list of school type to extract a list 
# of the number of students
school_type = byschool_summary['School ID'].count()
school_type

total_students = school_type.tolist()
total_students

total_students_1 = np.asarray(total_students)

# total_students_1 = total_students.np.asarray
# total_students_1

total_students_1
type(total_students_1)



# convert from list to np.array
# to run divisions


In [31]:
# Getting list of school budgets

school_budgets_pre = school_data_analysis.groupby(['school_name','budget'])
# school_budgets_pre.head(1)
school_budgets_calc = school_budgets_pre.size()
school_budgets_calc

school_budgets = pd.DataFrame(school_budgets_calc)
school_budgets.columns = school_budgets.columns.get_level_values(0)
school_budgets

school_name            budget 
Bailey High School     3124928    4976
Cabrera High School    1081356    1858
Figueroa High School   1884411    2949
Ford High School       1763916    2739
Griffin High School    917500     1468
Hernandez High School  3022020    4635
Holden High School     248087      427
Huang High School      1910635    2917
Johnson High School    3094650    4761
Pena High School       585858      962
Rodriguez High School  2547363    3999
Shelton High School    1056600    1761
Thomas High School     1043130    1635
Wilson High School     1319574    2283
Wright High School     1049400    1800
dtype: int64

,,0
school_name,budget,
Bailey High School,3124928,4976
Cabrera High School,1081356,1858
Figueroa High School,1884411,2949
Ford High School,1763916,2739
Griffin High School,917500,1468
Hernandez High School,3022020,4635
Holden High School,248087,427
Huang High School,1910635,2917
Johnson High School,3094650,4761


In [15]:
# Getting the budget spent on each student 



In [16]:
# s_avg_math_pre = byschool_summary['math_score'].mean()
# s_avg_math_pre
# s_avg_math = pd.DataFrame(s_avg_math_pre)
# s_avg_math

In [17]:
# s_avg_reading_pre = byschool_summary['reading_score'].mean()
# s_avg_reading_pre
# s_avg_reading = pd.DataFrame(s_avg_reading_pre)
# s_avg_reading

In [18]:
# s_pass_math = 
# s_pass_math

In [19]:
# s_pass_reading = 
# s_pass_reading

In [20]:
# need the appropriate conditional logic 

In [21]:
# % Passing Reading
# school_reading_pass_calc

In [22]:
# Overall Passing Rate (Average of the above two)

# use as guide 
# overall_pass_rate = round(((school_data_analysis['math_score'].mean()) + 
#                            (school_data_analysis['reading_score'].mean())) / 2, 2)
# overall_pass_rate


# school_overall_passing_rate = (school_math_pass + school_reading_pass) / 2


In [ ]:
# series into 

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [ ]:
# Top5Schools = school_summary_grouped.sort
# Top5Schools

## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [ ]:
# Bottom5Schools = school_summary_grouped.sort
# Bottom5Schools

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [ ]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

## Scores by School Type

* Perform the same operations as above, based on school type.